# Computing PageRank by Yourself (60%)

- In this assignment, we will ask you to implement the simplest PageRank algorithm (power iteration method) by yourself, which is a simple practise of NumPy.

- For better understanding of PageRank, you could refer to [PageRank, CUHK](http://home.ie.cuhk.edu.hk/~wkshum/papers/pagerank.pdf) and [PageRank, CMU](http://www.cs.cmu.edu/~elaw/pagerank.pdf).

## (1) Loading the dataset

We will use a small network (Bitcoin OTC trust weighted signed network) collected from a public dataset portal, [SNAP, Stanford](https://snap.stanford.edu/index.html).

In [2]:
import numpy as np
import pandas

data = pandas.read_csv("soc-sign-bitcoinotc.csv", names=['source', 'target', 'rating', 'time'])
# keep only records of high rating
print("number of total edges:", len(data))
data = data[data.rating>=8]
data=data.drop(['rating','time'],axis=1)
print("remained data shape", data.shape)
print("number of remained edges:", len(data))

max_idx = np.amax(data.values)
print("Maximum vertex ID is", max_idx)

# count the outgoing degree of every vertex
outdeg = np.zeros(max_idx+1)
for i in data.values:
    outdeg[i[0]]=outdeg[i[0]]+1

# construct the adjacent matrix
matrix = np.zeros((max_idx+1, max_idx+1))
for i in data.values:
    matrix[i[0],i[1]]=1/outdeg[i[0]]
    
# Think about why we need to transpose the matrix, refer to page 18 and 19 in the CMU slides
M = np.transpose(matrix)
# print(matrix)

number of total edges: 35592
remained data shape (1150, 2)
number of remained edges: 1150
Maximum vertex ID is 5999


## (2) Power iteration method
- For each iteration, you need to update the PageRank vector $\vec{v}$ by $\vec{v} = \alpha M \times \vec{v}$

In [3]:
def PowerMethod(v, M):
    # The implementation logic is the following
    # 1. use the matrix multiplication to calculate the new value of v, $new_v$
    # 2. if any change between new_v and v is larger than a threshold, then you go back to step (1); 
    #    e.g., if np.abs(new_v - v).any()>0.01:
    # 3. otherwise, the calculation finished and returns.
    
    count=0
    while True:
        if count>max_loop:
            break
        if count%1000==0:
            print("Calculating iteration #:", count)
        count=count+1

        # calculate new_v
        new_v = np.dot(M,v)
        
        # check convergence
        # the original test of any()>0.01 may not get the expected result
        if (np.abs(new_v - v)>0.01).any():
            v=new_v
            continue
        else:
            break
    
    return new_v
    

# (3) Handling Dangling Node
- A node is called a dangling node if it does not contain any out-going link, i.e., if the out-degree is zero.
- Read [PageRank, CUHK](http://home.ie.cuhk.edu.hk/~wkshum/papers/pagerank.pdf) for more information.

$$ MD_{ij}=\left\{
\begin{aligned}
1/deg(j) &  & \text{if there a link from node $j$ to node $i$} \\
1/N  &  & \text{if vertex $j$ is a dangling node} \\
0 & & \text{otherwise}
\end{aligned}
\right.$$

In [4]:
# You need to calculate MD baesd on M
N = max_idx+1
D = np.zeros((N,N))
for j in range(N):
    if outdeg[j]==0:
        D[:,j]=1/N
MD = M + D

# (4) Handling Reducible Web Graph

- To resolve reducible web navigation (that is the loop in between some nodes)
    - Read [PageRank, CUHK](http://home.ie.cuhk.edu.hk/~wkshum/papers/pagerank.pdf) for more information.
- We further update the Matrix MD to MR

$$
MR = \alpha MD + \frac{1-\alpha}{N}\begin{bmatrix} 
1 & ... & 1 \\
... & ... & ... \\
1 & ... & 1
\end{bmatrix}\quad
$$

where $\alpha$ is set to 0.85

In [5]:
# You need to update MR based on MD
a = 0.85
A = np.ones((N,N))
MR = a*MD+(((1-a)/N)*A)

# (5) Main program

In [36]:
def showtopk(result):
    # generate the top-5 result
    # enhanced to support investors with same scores
    sorted_index = np.argsort(result)
    top5_investors = sorted_index[:-6:-1]
    least5_investors = sorted_index[:5]
    top5_score = [result[i] for i in top5_investors]
    least5_score = [result[i] for i in least5_investors]
    top5 = zip(top5_investors, top5_score)
    least5 = zip(least5_investors, least5_score)

    print("Top 5 best-rated players and their scores are:")
    for investor, score in top5: print("- investidor %d, com score: %.5f" % (investor, score))
    print("Last 5 least-rated players and their scores are:")
    for investor, score in least5: print("- investidor %d, com score: %.5f" % (investor, score))

max_loop=100

# initialize the vector $v$
v=np.random.random((max_idx+1,))
# using 1/N  as the initial vector would get the convergent result faster and tend to have the same top 5
# v = np.ones((N,1))
# v = v/N

# call the function at step (2)
print("[CALCULATING M's PageRank]")
result = list(PowerMethod(v, M))
showtopk(result)

# call the function at step (3)
print("[CALCULATING MD's PageRank]")
result_D = list(PowerMethod(v, MD))
showtopk(result_D)

# call the function at step (4)
print("[CALCULATING MR's PageRank]")
result_R = list(PowerMethod(v, MR))
showtopk(result_R)

[CALCULATING M's PageRank]
Calculating iteration #: 0
Top 5 best-rated players and their scores are:
- investidor 202, com score: 3.02077
- investidor 144, com score: 1.81948
- investidor 361, com score: 1.61003
- investidor 5029, com score: 1.23645
- investidor 3996, com score: 1.14969
Last 5 least-rated players and their scores are:
- investidor 0, com score: 0.00000
- investidor 3965, com score: 0.00000
- investidor 3964, com score: 0.00000
- investidor 3963, com score: 0.00000
- investidor 3962, com score: 0.00000
[CALCULATING MD's PageRank]
Calculating iteration #: 0
Top 5 best-rated players and their scores are:
- investidor 202, com score: 91.69217
- investidor 144, com score: 90.31732
- investidor 361, com score: 60.71600
- investidor 3996, com score: 59.96146
- investidor 2567, com score: 37.31499
Last 5 least-rated players and their scores are:
- investidor 0, com score: 0.08137
- investidor 3937, com score: 0.08137
- investidor 3936, com score: 0.08137
- investidor 3935, com

# Written Part (40%)
1. What's the broadcasting feature in NumPy? Please explain your understanding and give some example how it is useful in applications.

2. What will happen if you don't use vectorized calculation in the PowerMethod? 

3. What's the problem of our current PowerMethod implementation? Please refer to the CUHK slides and answer this question.

1. Arrays with different sizes cannot be added, subtracted, or generally be used in arithmetic. A way to overcome this is to duplicate the smaller array so that it is the dimensionality and size as the larger array. This is called array broadcasting and is available in NumPy when performing array arithmetic, which can greatly reduce and simplify your code. For example, we can add scalar 5 to a two-dimensional array by broadcasting the addition of 5 to each value of the two-dimensional array. Below is a working example:

In [6]:
# a 2x3 array
myArray=np.array([[1,2,3],[4,5,6]])
# to add 5 to each value of this array, without boardcasting, 
# we might need to either loop through all values on the array one by one and add 5 to each of them
# or create another 2x3 array with all values set to 5
fiveArray=np.array([[5,5,5],[5,5,5]])
print(myArray+fiveArray)
# with boardcasting, we smiply add the scalar 5 to it and will get the same result
print(myArray+5)

[[ 6  7  8]
 [ 9 10 11]]
[[ 6  7  8]
 [ 9 10 11]]


2. Without vectorized calculation in PowerMethod, we would have to loop through each value of vector v with the corresponding row of matrix M. Loop through the corresponding row of M to calculate the sum product of corresponding value of v to get the correspnding new value of new_v. With the vectorized calculation, we can simply write this in one line of code with the direct mapping to the given vector formular.

3. Many vertex/investidor/pages may have been ranked as equal scores, i.e. they are not all distinct. This can be seen from the last 5 scores analysis as added in showtopk() function

# Bonus (10%)
- Revise the PowerMethod and make it support Personalized PageRank

We don't have to modify the PowerMethod to support Personalized PageRank. We can model all persons and their preferred pages as nodes to create a bi-directional graph between persons and pages. We further create a link from a particular person loop back to himself/herself. This particular person's Personalized PageRank will then be calculated by the corresponding MR matrix.